In [4]:
import cdsapi

def data_download(year, month, region):

    c = cdsapi.Client()
    c.retrieve(
        'satellite-fire-burned-area',
        {
            'origin': 'esa_cci',
            'sensor': 'modis',
            'variable': 'pixel_variables',
            'version': '5_1_1cds',
            'region': f'{region}',
            'year': f'{year}',
            'month': f'{str(month).zfill(2)}',
            'nominal_day': '01',
            'format': 'tgz',
        },
        f'../data/{region}_{year}_{str(month).zfill(2)}.tar.gz')
       
    return unpack_data()


In [5]:
def unpack_data():
    import os
    import glob
    data_dir = '../data'

# Find all .tar.gz files in the data directory
    tar_files = glob.glob(os.path.join(data_dir, '*.tar.gz'))

# Unpack and remove each .tar.gz file
    for tar_file in tar_files:
        os.system(f'tar -xf {tar_file} -C {data_dir}')
        os.remove(tar_file)

In [6]:
#just select date and region
#modifying forloop should not be necessary

start_year = 2001
end_year = 2001
start_month = 8
end_month = 8
region = 'europe'

for i in range(start_year, end_year + 1):
    for j in range(start_month, end_month + 1):
        data_download(i, j, region)

2023-06-07 11:19:24,218 INFO Welcome to the CDS
2023-06-07 11:19:24,220 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/satellite-fire-burned-area
2023-06-07 11:19:24,278 INFO Request is completed
2023-06-07 11:19:24,280 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data6/dataset-satellite-fire-burned-area-13ece09a-4791-417c-a9bb-43f5d19dbb5f.tar.gz to ../data/europe_2001_08.tar.gz (47.8M)
2023-06-07 11:19:29,217 INFO Download rate 9.7M/s   


In [1]:
import xarray as xr
import pandas as pd
import numpy as np

pixel_ds = xr.open_dataset('../data/20010801-ESACCI-L3S_FIRE-BA-MODIS-AREA_3-fv5.1.1cds.nc', engine='netcdf4')
pixel_ds

<xarray.Dataset>
Dimensions:      (time: 1, lat: 25827, lon: 35178, bounds: 2)
Coordinates:
  * lon          (lon) float64 -26.0 -26.0 -26.0 -25.99 ... 52.99 53.0 53.0
  * lat          (lat) float64 83.0 83.0 82.99 82.99 ... 25.01 25.01 25.0 25.0
  * time         (time) datetime64[ns] 2001-07-31
Dimensions without coordinates: bounds
Data variables:
    JD           (time, lat, lon) int16 ...
    CL           (time, lat, lon) int8 ...
    LC           (time, lat, lon) uint8 ...
    lat_bounds   (lat, bounds) float64 ...
    lon_bounds   (lon, bounds) float64 ...
    time_bounds  (time, bounds) datetime64[ns] ...
Attributes: (12/38)
    title:                      Fire_cci Pixel MODIS Burned Area product
    institution:                University of Alcala
    source:                     MODIS MOD09GQ Collection 6, MODIS MOD09GA Col...
    history:                    Created on 2017-12-19 06:42:41; modified with...
    references:                 See www.esa-fire-cci.org
    tracking_id:                6cb63d40-88fe-4e50-a3aa-d66bb1474470
    ...                         ...
    geospatial_lon_units:       degrees_east
    geospatial_lat_units:       degrees_north
    geospatial_lon_resolution:  0.0022457331
    geospatial_lat_resolution:  0.0022457331
    product_version:            v5.1.1cds
    id:                         20010801-ESACCI-L3S_FIRE-BA-MODIS-AREA_3-fv5....

In [2]:
#pixel_ds = pixel_ds.squeeze('time')
def round_to_nearest_025(x):
    return np.round(x * 4) / 4

pixel_ds['lat'] = xr.apply_ufunc(round_to_nearest_025, pixel_ds['lat'])
pixel_ds['lon'] = xr.apply_ufunc(round_to_nearest_025, pixel_ds['lon'])

pixel_ds


<xarray.Dataset>
Dimensions:      (time: 1, lat: 25827, lon: 35178, bounds: 2)
Coordinates:
  * lon          (lon) float64 -26.0 -26.0 -26.0 -26.0 ... 53.0 53.0 53.0 53.0
  * lat          (lat) float64 83.0 83.0 83.0 83.0 83.0 ... 25.0 25.0 25.0 25.0
  * time         (time) datetime64[ns] 2001-07-31
Dimensions without coordinates: bounds
Data variables:
    JD           (time, lat, lon) int16 ...
    CL           (time, lat, lon) int8 ...
    LC           (time, lat, lon) uint8 ...
    lat_bounds   (lat, bounds) float64 ...
    lon_bounds   (lon, bounds) float64 ...
    time_bounds  (time, bounds) datetime64[ns] ...
Attributes: (12/38)
    title:                      Fire_cci Pixel MODIS Burned Area product
    institution:                University of Alcala
    source:                     MODIS MOD09GQ Collection 6, MODIS MOD09GA Col...
    history:                    Created on 2017-12-19 06:42:41; modified with...
    references:                 See www.esa-fire-cci.org
    tracking_id:                6cb63d40-88fe-4e50-a3aa-d66bb1474470
    ...                         ...
    geospatial_lon_units:       degrees_east
    geospatial_lat_units:       degrees_north
    geospatial_lon_resolution:  0.0022457331
    geospatial_lat_resolution:  0.0022457331
    product_version:            v5.1.1cds
    id:                         20010801-ESACCI-L3S_FIRE-BA-MODIS-AREA_3-fv5....

In [3]:
filtered_data = pixel_ds.squeeze('time')
filtered_data = filtered_data.sel(lat=slice(52, 33), lon=slice(-10, 50))
filtered_data = filtered_data[['CL',
                               'lat_bounds',
                               'lon_bounds'
                               ]]

filtered_data = filtered_data.drop_duplicates(dim=['lat','lon'])




filtered_data

<xarray.Dataset>
Dimensions:     (lat: 77, lon: 241, bounds: 2)
Coordinates:
  * lon         (lon) float64 -10.0 -9.75 -9.5 -9.25 ... 49.25 49.5 49.75 50.0
  * lat         (lat) float64 52.0 51.75 51.5 51.25 ... 33.75 33.5 33.25 33.0
    time        datetime64[ns] 2001-07-31
Dimensions without coordinates: bounds
Data variables:
    CL          (lat, lon) int8 ...
    lat_bounds  (lat, bounds) float64 ...
    lon_bounds  (lon, bounds) float64 ...
Attributes: (12/38)
    title:                      Fire_cci Pixel MODIS Burned Area product
    institution:                University of Alcala
    source:                     MODIS MOD09GQ Collection 6, MODIS MOD09GA Col...
    history:                    Created on 2017-12-19 06:42:41; modified with...
    references:                 See www.esa-fire-cci.org
    tracking_id:                6cb63d40-88fe-4e50-a3aa-d66bb1474470
    ...                         ...
    geospatial_lon_units:       degrees_east
    geospatial_lat_units:       degrees_north
    geospatial_lon_resolution:  0.0022457331
    geospatial_lat_resolution:  0.0022457331
    product_version:            v5.1.1cds
    id:                         20010801-ESACCI-L3S_FIRE-BA-MODIS-AREA_3-fv5....

In [4]:
pixel_df = filtered_data.to_dataframe().reset_index()
pixel_df = pixel_df.drop(['lat_bounds', 'lon_bounds', 'bounds'], axis = 1)

pixel_df

,lat,lon,CL,time
0,52.0,-10.00,1,2001-07-31
1,52.0,-10.00,1,2001-07-31
2,52.0,-9.75,0,2001-07-31
3,52.0,-9.75,0,2001-07-31
4,52.0,-9.50,1,2001-07-31
...,...,...,...,...
37109,33.0,49.50,1,2001-07-31
37110,33.0,49.75,1,2001-07-31
37111,33.0,49.75,1,2001-07-31
37112,33.0,50.00,1,2001-07-31


In [5]:
pixel_df['year'] = pixel_df['time'].dt.year
pixel_df['month'] = pixel_df['time'].dt.month
pixel_df = pixel_df.drop('time', axis= 1).dropna()
pixel_df

,lat,lon,CL,year,month
0,52.0,-10.00,1,2001,7
1,52.0,-10.00,1,2001,7
2,52.0,-9.75,0,2001,7
3,52.0,-9.75,0,2001,7
4,52.0,-9.50,1,2001,7
...,...,...,...,...,...
37109,33.0,49.50,1,2001,7
37110,33.0,49.75,1,2001,7
37111,33.0,49.75,1,2001,7
37112,33.0,50.00,1,2001,7


In [3]:
import xarray as xr
import pandas as pd
import numpy as np
import os


def round_to_nearest_025(x):
    return np.round(x * 4) / 4

def pixel_preprocessing(file_path):
    pixel_ds = xr.open_dataset(file_path, engine='netcdf4')

    pixel_ds['lat'] = xr.apply_ufunc(round_to_nearest_025, pixel_ds['lat'])
    pixel_ds['lon'] = xr.apply_ufunc(round_to_nearest_025, pixel_ds['lon'])

    filtered_data = pixel_ds.squeeze('time')
    filtered_data = filtered_data.sel(lat=slice(52, 33), lon=slice(-10, 50))
    filtered_data = filtered_data[['CL',
                                   'lat_bounds',
                                   'lon_bounds'
                                   ]]

    filtered_data = filtered_data.drop_duplicates(dim=['lat','lon'])

    pixel_df = filtered_data.to_dataframe().reset_index()
    pixel_df = pixel_df.drop(['lat_bounds', 'lon_bounds', 'bounds'], axis = 1)

    pixel_df['year'] = pixel_df['time'].dt.year
    pixel_df['month'] = pixel_df['time'].dt.month
    pixel_df = pixel_df.drop('time', axis= 1).dropna()
    return pixel_df

In [2]:
pixel_preprocessing('../data/20010801-ESACCI-L3S_FIRE-BA-MODIS-AREA_3-fv5.1.1cds.nc')

,lat,lon,CL,year,month
0,52.0,-10.00,1,2001,7
1,52.0,-10.00,1,2001,7
2,52.0,-9.75,0,2001,7
3,52.0,-9.75,0,2001,7
4,52.0,-9.50,1,2001,7
...,...,...,...,...,...
37109,33.0,49.50,1,2001,7
37110,33.0,49.75,1,2001,7
37111,33.0,49.75,1,2001,7
37112,33.0,50.00,1,2001,7


In [ ]:
data_dir = '../data'
dfs_grid = []
dfs_pixel = []

for file_name in os.listdir(data_dir):
    if file_name.endswith('.nc') and 'FIRE' in file_name:
        if 'AREA' in file_name:
            file_path = os.path.join(data_dir, file_name)
            df_pixel = pixel_preprocessing(file_path)
            dfs_pixel.append(df_pixel)
        
        else:
            file_path = os.path.join(data_dir, file_name)
            df_grid = fire_preprocessing(file_path)            
            dfs_grid.append(df_grid)

big_pixel_df = pd.concat(dfs_pixel, ignore_index=True)
big_grid_df = pd.concat(dfs_grid, ignore_index=True)

final_df = big_pixel_df.merge(
    big_grid_df,
    how='left',
    left_on=['lat', 'lon', 'year', 'month'],
    right_on=['lat_bounds', 'lon_bounds', 'year', 'month']
    )

final_df.to_csv(os.path.join(data_dir, 'fire.csv'), index=False)